## <strong>VGG 16을 활용한 trash 분류 (Transfer Learning 활용)</strong>

- kaggle dataset(https://www.kaggle.com/datasets/asdasdasasdas/garbage-classification)을 활용한 trash 분류 모델

## 순서

1. **import**: 필요한 모듈 import
2. **전처리**: 학습에 필요한 데이터 전처리를 수행합니다.
3. **모델링(model)**: 모델을 정의합니다.
4. **컴파일(compile)**: 모델을 생성합니다.
5. **학습 (fit)**: 모델을 학습시킵니다.

### 1) **import**: 필요한 모듈 import

In [1]:
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, GlobalAveragePooling2D, Flatten, Dropout, Dense, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy

### 2) **전처리**: 학습에 필요한 데이터 전처리를 수행합니다.

In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

path='/content/gdrive/My Drive/AI/dataset/garbage-classification/Garbage-classification/Garbage-classification'

train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input,
    rotation_range=30,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.1,
    horizontal_flip = True,
    fill_mode = 'nearest',
)

train_generator = train_datagen.flow_from_directory(
    directory=path, 
    target_size=(224,224), 
    classes=['cardboard', 'glass', 'metal','paper', 'plastic', 'trash'], 
    batch_size=16, 
    subset='training'
)

valid_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input,
    validation_split=0.1,
    rescale=1./255,
)


validation_generator = valid_datagen.flow_from_directory(
    directory=path, 
    target_size=(224,224), 
    classes=['cardboard', 'glass', 'metal','paper', 'plastic', 'trash'], 
    batch_size=16, 
    subset='validation'
)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Found 2276 images belonging to 6 classes.
Found 251 images belonging to 6 classes.


### 3) **모델링(model)**: 모델을 정의합니다.

In [8]:
transfer_model = VGG16(weights='imagenet', include_top = False, input_shape=(224,224,3))
transfer_model.trainable = False

model = Sequential([
  transfer_model,
  GlobalAveragePooling2D(),
  Dense(512, activation='relu'),
  BatchNormalization(),
  Dropout(0.3),
  Dense(128, activation='relu'),
  BatchNormalization(),
  Dropout(0.3),
  Dense(6, activation='softmax')
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d_1   (None, 512)              0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_3 (Dense)             (None, 512)               262656    
                                                                 
 batch_normalization_2 (Batc  (None, 512)              2048      
 hNormalization)                                                 
                                                                 
 dropout_2 (Dropout)         (None, 512)               0         
                                                                 
 dense_4 (Dense)             (None, 128)              

### 4) **컴파일(compile)**: 모델을 생성합니다.

In [9]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['acc'])

checkpoint_path = "garbage-classifiction-checkpoint.ckpt"
checkpoint = ModelCheckpoint(filepath=checkpoint_path, 
                             save_weights_only=True, 
                             save_best_only=True, 
                             monitor='val_loss', 
                             verbose=1)

### 5) **학습 (fit)**: 모델을 학습시킵니다.

In [10]:
model.fit(train_generator,
          validation_data=(validation_generator),
          epochs=30,
          callbacks=[checkpoint]
          )

model.load_weights(checkpoint_path)
model.save('garbage-classification-model1(val_split=0.1).h5')

Epoch 1/30
143/143 [==============================] - ETA: 0s - loss: 2.0110 - acc: 0.3106
Epoch 1: val_loss improved from inf to 1.60834, saving model to garbage-classifiction-checkpoint.ckpt
143/143 [==============================] - 464s 3s/step - loss: 2.0110 - acc: 0.3106 - val_loss: 1.6083 - val_acc: 0.3347
Epoch 2/30
143/143 [==============================] - ETA: 0s - loss: 1.4839 - acc: 0.4521
Epoch 2: val_loss improved from 1.60834 to 1.25787, saving model to garbage-classifiction-checkpoint.ckpt
143/143 [==============================] - 41s 284ms/step - loss: 1.4839 - acc: 0.4521 - val_loss: 1.2579 - val_acc: 0.5458
Epoch 3/30
143/143 [==============================] - ETA: 0s - loss: 1.3038 - acc: 0.5299
Epoch 3: val_loss improved from 1.25787 to 1.00946, saving model to garbage-classifiction-checkpoint.ckpt
143/143 [==============================] - 41s 283ms/step - loss: 1.3038 - acc: 0.5299 - val_loss: 1.0095 - val_acc: 0.6096
Epoch 4/30
143/143 [=======================

### 모델2) 

In [11]:
transfer_model = VGG16(weights='imagenet', include_top = False, input_shape=(224,224,3))
transfer_model.trainable = False

model_2 = Sequential([
  transfer_model,
  Flatten(),
  Dropout(0.5),
  Dense(512, activation='relu'),
  Dense(128, activation='relu'),
  Dense(6, activation='softmax')
])

model_2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dropout_4 (Dropout)         (None, 25088)             0         
                                                                 
 dense_6 (Dense)             (None, 512)               12845568  
                                                                 
 dense_7 (Dense)             (None, 128)               65664     
                                                                 
 dense_8 (Dense)             (None, 6)                 774       
                                                                 
Total params: 27,626,694
Trainable params: 12,912,006


### 4) **모델2 컴파일(compile)**: 모델을 생성합니다.

In [14]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['acc'])

checkpoint_path = "garbage-classifiction-checkpoint.ckpt"
checkpoint = ModelCheckpoint(filepath=checkpoint_path, 
                             save_weights_only=True, 
                             save_best_only=True, 
                             monitor='val_loss', 
                             verbose=1)

### 5) **모델2 학습 (fit)**: 모델을 학습시킵니다.

In [15]:
model_2.fit(train_generator,
          validation_data=(validation_generator),
          epochs=30,
          callbacks=[checkpoint]
          )

model_2.load_weights(checkpoint_path)
model_2.save('garbage-classification-model2.h5')

Epoch 1/30
143/143 [==============================] - ETA: 0s - loss: 1.3443 - acc: 0.4758
Epoch 1: val_loss improved from inf to 1.24916, saving model to garbage-classifiction-checkpoint.ckpt
143/143 [==============================] - 40s 280ms/step - loss: 1.3443 - acc: 0.4758 - val_loss: 1.2492 - val_acc: 0.5378
Epoch 2/30
143/143 [==============================] - ETA: 0s - loss: 1.0376 - acc: 0.6098
Epoch 2: val_loss improved from 1.24916 to 0.81966, saving model to garbage-classifiction-checkpoint.ckpt
143/143 [==============================] - 39s 275ms/step - loss: 1.0376 - acc: 0.6098 - val_loss: 0.8197 - val_acc: 0.6813
Epoch 3/30
143/143 [==============================] - ETA: 0s - loss: 0.8970 - acc: 0.6446
Epoch 3: val_loss improved from 0.81966 to 0.68535, saving model to garbage-classifiction-checkpoint.ckpt
143/143 [==============================] - 39s 273ms/step - loss: 0.8970 - acc: 0.6446 - val_loss: 0.6853 - val_acc: 0.7371
Epoch 4/30
143/143 [=====================